In [218]:
import pandas as pd
import ast
import json

In [107]:
# pasamos los archivos a un Dataframe

#output
dfoutput=pd.read_json(r'output_steam_games.json',lines=True, encoding='MacRoman')

#reviews
with open('australian_user_reviews.json',encoding='MacRoman') as archivo_json:
    contenido_json = archivo_json.readlines()
lisreviews=list(map(ast.literal_eval,contenido_json))
dfreviews=pd.DataFrame(lisreviews)

#items
with open('australian_users_items.json',encoding='MacRoman') as archivo_json:
    contenido_json = archivo_json.readlines()
lisitems=[eval(line.strip())for line in contenido_json]
dfitems=pd.DataFrame(lisitems)


In [108]:
dfoutput.tail()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"
120444,None,None,Maze Run VR,None,http://store.steampowered.com/app/681550/Maze_...,None,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,1.0,681550.0,None


In [145]:
#conservamos las columnas que nos van a servir para nuestros objetivos
dfjuegos=pd.DataFrame(dfoutput[['id','app_name','developer','publisher','release_date']].drop_duplicates()[1:])

In [146]:
dfjuegos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32133 entries, 88310 to 120444
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            32132 non-null  float64
 1   app_name      32132 non-null  object 
 2   developer     28835 non-null  object 
 3   publisher     24082 non-null  object 
 4   release_date  30067 non-null  object 
dtypes: float64(1), object(4)
memory usage: 1.5+ MB


In [147]:
dfjuegos[dfjuegos.id.isnull()]

,id,app_name,developer,publisher,release_date
119271,NaN,Batman: Arkham City - Game of the Year Edition,"Rocksteady Studios,Feral Interactive (Mac)","Warner Bros. Interactive Entertainment, Feral ...",2012-09-07


In [148]:
dfjuegos[dfjuegos.app_name.isnull()]

,id,app_name,developer,publisher,release_date
90890,317160.0,None,None,None,2014-08-26


In [150]:
# sustituimos valores nulos de la columna desarrolladores con la columna de publihser, solo en las filas donde no hya nulos en publisher
dfjuegos.loc[(dfjuegos.developer.isnull())&(~dfjuegos.publisher.isnull()),'developer']=dfjuegos.publisher

In [151]:
#quitamos la columna publisher, ya que solo nos interesa la de desarrollador
dfjuegos.drop(columns='publisher',inplace=True)
dfjuegos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32133 entries, 88310 to 120444
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            32132 non-null  float64
 1   app_name      32132 non-null  object 
 2   developer     28900 non-null  object 
 3   release_date  30067 non-null  object 
dtypes: float64(1), object(3)
memory usage: 2.2+ MB


In [152]:
#sacamos el año de lanzamiento de la columna release_date y quitamos la columna release_date
dfjuegos['Año_lanzamiento']=dfjuegos.release_date.str.split('-',expand=True,n=1).drop(columns=1)
dfjuegos.drop(columns='release_date', inplace=True)

In [153]:
#renombramos los nombres de las columnas
dfjuegos.rename(columns={'Año lanzamiento':'Año_lanzamiento','app_name':'Nombre','developer':'Desarrollador','id':'Id'},inplace=True)

In [154]:
#hacemos una lista de los años que hay en la columna Año_de_lanzamiento
años=[]
for a in dfjuegos.Año_lanzamiento.sort_values().unique():
    try:
        int(a)
    except:
        continue
    else:
        años.append(a)
años[0:5]

['1970', '1975', '1980', '1981', '1982']

In [155]:
#cambiamos las filas que tienen el año en una parte de su cadena a contener solo el dato del año
for a in años:
    dfjuegos.loc[dfjuegos.Año_lanzamiento.str.contains(a,na=False),'Año_lanzamiento']=a

In [156]:
dfjuegos.Año_lanzamiento.sort_values().unique()

array(['"""Soon"""',
       '0̵1̴0̵0̶1̷0̶0̵0̴ ̴0̶0̶1̶1̶0̷0̶1̵1̴ ̸0̶0̶1̶1̵0̶1̷0̴0̵ ̴0̶1̷0̸1̵0̷0̴1̶0̴ ̴0̷0̴1̷1̶0̶1̵1̷1̵ ̵',
       '14 July', '1970', '1975', '1980', '1981', '1982', '1983', '1984',
       '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992',
       '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '2019', '2021', 'Beta测试已开启',
       "C'est bientôt...                    (ou pas)", 'Coming Soon',
       'Coming Soon!', 'Coming Soon/Próximamente', 'Coming soon',
       'Coming soon..', 'Comming Soon',
       'Datachunks conflicted. Be vigiliant.', 'Demo coming soon.',
       'Demo is available now!', 'Early Access Starting Soon!',
       'Early Access soon', 'Not yet available', 'Play Beta in demo!',
       'Please wait warmly', 'Release Date TBA', 'SOON', 'SOON™', 'Soon',
       'Soon..'

In [157]:
# se hace una lista de los datos que asiguen pendientes en la columna año de lanzamiento, y haciendo investigacion se encuentra que los juegos fueron lanzados del 2018 en adelante
datossinaño=[]
for a in dfjuegos.Año_lanzamiento.sort_values().unique():
    if a not in años:
        datossinaño.append(a)
datossinaño

['"""Soon"""',
 '0̵1̴0̵0̶1̷0̶0̵0̴ ̴0̶0̶1̶1̶0̷0̶1̵1̴ ̸0̶0̶1̶1̵0̶1̷0̴0̵ ̴0̶1̷0̸1̵0̷0̴1̶0̴ ̴0̷0̴1̷1̶0̶1̵1̷1̵ ̵',
 '14 July',
 'Beta测试已开启',
 "C'est bientôt...                    (ou pas)",
 'Coming Soon',
 'Coming Soon!',
 'Coming Soon/Próximamente',
 'Coming soon',
 'Coming soon..',
 'Comming Soon',
 'Datachunks conflicted. Be vigiliant.',
 'Demo coming soon.',
 'Demo is available now!',
 'Early Access Starting Soon!',
 'Early Access soon',
 'Not yet available',
 'Play Beta in demo!',
 'Please wait warmly',
 'Release Date TBA',
 'SOON',
 'SOON™',
 'Soon',
 'Soon..',
 'Soon™',
 'TBA',
 'TBD',
 'To Be Announced',
 'To be Announced',
 'To be announced',
 'When it is finished',
 'When it is ready',
 "When it's done",
 "When it's done!",
 'coming soon',
 'early access',
 'soon',
 '预热群52756441',
 None]

In [158]:
#falta comprobar si los videjugos que no tienen año son relevantes para las funciones y en caso que no, hare la conversion despues al año 2018 que es al año en que la mayoria se lanzaron

In [202]:
# unimos los generos al dataframe de juegos
dfjuegos=dfjuegos.join(dfoutput[['genres','id']],how='inner')


In [204]:
dfjuegos.head()

,Id,Nombre,Desarrollador,Año_lanzamiento,genres,id
88310,761140.0,Lost Summoner Kitty,Kotoshiro,2018,"[Action, Casual, Indie, Simulation, Strategy]",761140.0
88311,643980.0,Ironbound,Secret Level SRL,2018,"[Free to Play, Indie, RPG, Strategy]",643980.0
88312,670290.0,Real Pool 3D - Poolians,Poolians.com,2017,"[Casual, Free to Play, Indie, Simulation, Sports]",670290.0
88313,767400.0,弹炸人2222,彼岸领域,2017,"[Action, Adventure, Casual]",767400.0
88314,773570.0,Log Challenge,None,None,None,773570.0


In [257]:
#guardamos en un csv para guardar los avances y partir de ese punto
dfjuegos.to_csv('Juegos_steam.csv',index=False)

In [259]:
# creamos una lista con los generos existentes
generos=[]
for a in dfjuegos.genres.dropna().drop_duplicates():
    for b in a:
        if b not in generos:
            generos.append(b)
generos=sorted(generos)
print(generos)

['Accounting', 'Action', 'Adventure', 'Animation &amp; Modeling', 'Audio Production', 'Casual', 'Design &amp; Illustration', 'Early Access', 'Education', 'Free to Play', 'Indie', 'Massively Multiplayer', 'Photo Editing', 'RPG', 'Racing', 'Simulation', 'Software Training', 'Sports', 'Strategy', 'Utilities', 'Video Production', 'Web Publishing']


In [208]:
dfitems.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [231]:
# Se desanida la columna de items y se pasa a un Dataframe el cual contiene la informacion de cada juego jugado por el usuario, como horas jugadas y el id del juego
lista_items=[]
for a in dfitems['items']:
    lista_items.append(ast.literal_eval(str(a)))
dfitems2=pd.DataFrame(lista_items)

In [254]:
#hacemos un dataframe con los reviews y los id de los usuarios
dfitems2=pd.concat([dfitems.user_id,dfitems2],axis=1)
dfitems2.head()

,user_id,0,1,2,3,4,5,6,7,8,...,7752,7753,7754,7755,7756,7757,7758,7759,7760,7761
0,76561197970982479,"{'item_id': '10', 'item_name': 'Counter-Strike...","{'item_id': '20', 'item_name': 'Team Fortress ...","{'item_id': '30', 'item_name': 'Day of Defeat'...","{'item_id': '40', 'item_name': 'Deathmatch Cla...","{'item_id': '50', 'item_name': 'Half-Life: Opp...","{'item_id': '60', 'item_name': 'Ricochet', 'pl...","{'item_id': '70', 'item_name': 'Half-Life', 'p...","{'item_id': '130', 'item_name': 'Half-Life: Bl...","{'item_id': '300', 'item_name': 'Day of Defeat...",...,None,None,None,None,None,None,None,None,None,None
1,js41637,"{'item_id': '10', 'item_name': 'Counter-Strike...","{'item_id': '80', 'item_name': 'Counter-Strike...","{'item_id': '100', 'item_name': 'Counter-Strik...","{'item_id': '300', 'item_name': 'Day of Defeat...","{'item_id': '30', 'item_name': 'Day of Defeat'...","{'item_id': '40', 'item_name': 'Deathmatch Cla...","{'item_id': '60', 'item_name': 'Ricochet', 'pl...","{'item_id': '240', 'item_name': 'Counter-Strik...","{'item_id': '280', 'item_name': 'Half-Life: So...",...,None,None,None,None,None,None,None,None,None,None
2,evcentric,"{'item_id': '1200', 'item_name': 'Red Orchestr...","{'item_id': '1230', 'item_name': 'Mare Nostrum...","{'item_id': '1280', 'item_name': 'Darkest Hour...","{'item_id': '1520', 'item_name': 'DEFCON', 'pl...","{'item_id': '220', 'item_name': 'Half-Life 2',...","{'item_id': '320', 'item_name': 'Half-Life 2: ...","{'item_id': '340', 'item_name': 'Half-Life 2: ...","{'item_id': '360', 'item_name': 'Half-Life Dea...","{'item_id': '380', 'item_name': 'Half-Life 2: ...",...,None,None,None,None,None,None,None,None,None,None
3,Riot-Punch,"{'item_id': '10', 'item_name': 'Counter-Strike...","{'item_id': '20', 'item_name': 'Team Fortress ...","{'item_id': '30', 'item_name': 'Day of Defeat'...","{'item_id': '40', 'item_name': 'Deathmatch Cla...","{'item_id': '50', 'item_name': 'Half-Life: Opp...","{'item_id': '60', 'item_name': 'Ricochet', 'pl...","{'item_id': '70', 'item_name': 'Half-Life', 'p...","{'item_id': '130', 'item_name': 'Half-Life: Bl...","{'item_id': '80', 'item_name': 'Counter-Strike...",...,None,None,None,None,None,None,None,None,None,None
4,doctr,"{'item_id': '300', 'item_name': 'Day of Defeat...","{'item_id': '20', 'item_name': 'Team Fortress ...","{'item_id': '50', 'item_name': 'Half-Life: Opp...","{'item_id': '70', 'item_name': 'Half-Life', 'p...","{'item_id': '130', 'item_name': 'Half-Life: Bl...","{'item_id': '10', 'item_name': 'Counter-Strike...","{'item_id': '30', 'item_name': 'Day of Defeat'...","{'item_id': '40', 'item_name': 'Deathmatch Cla...","{'item_id': '60', 'item_name': 'Ricochet', 'pl...",...,None,None,None,None,None,None,None,None,None,None


In [262]:
# Se guarda en un csv el dataframe con los items desanidados y el id de usuario
dfitems2.to_csv('items_por_usuario.csv',index=False)

In [161]:
dfreviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [253]:
#desanidamos la columna items, se crean diccionaros dentro de cada celda donde se puede disponer de la info accediendo al diccionario, y se agrega el id del usuario en el dataframe
lista_reviews=[]
for a in dfreviews.reviews:
    lista_reviews.append(ast.literal_eval(str(a)))
dfreviews2=pd.concat([dfreviews.user_id,pd.DataFrame(lista_reviews)],axis=1)
dfreviews2

,user_id,0,1,2,3,4,5,6,7,8,9
0,76561197970982479,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None
1,js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....","{'funny': '', 'posted': 'Posted September 8, 2...","{'funny': '', 'posted': 'Posted November 29, 2...",None,None,None,None,None,None,None
2,evcentric,"{'funny': '', 'posted': 'Posted February 3.', ...","{'funny': '', 'posted': 'Posted December 4, 20...","{'funny': '', 'posted': 'Posted November 3, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...",None,None,None,None
3,doctr,"{'funny': '', 'posted': 'Posted October 14, 20...","{'funny': '', 'posted': 'Posted July 28, 2012....","{'funny': '', 'posted': 'Posted June 2, 2012.'...","{'funny': '', 'posted': 'Posted June 29, 2014....","{'funny': '', 'posted': 'Posted November 22, 2...","{'funny': '', 'posted': 'Posted February 23, 2...",None,None,None,None
4,maplemage,"{'funny': '3 people found this review funny', ...","{'funny': '1 person found this review funny', ...","{'funny': '2 people found this review funny', ...","{'funny': '', 'posted': 'Posted July 11, 2013....",None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
25794,76561198306599751,"{'funny': '', 'posted': 'Posted May 31.', 'las...",None,None,None,None,None,None,None,None,None
25795,Ghoustik,"{'funny': '', 'posted': 'Posted June 17.', 'la...",None,None,None,None,None,None,None,None,None
25796,76561198310819422,"{'funny': '1 person found this review funny', ...",None,None,None,None,None,None,None,None,None
25797,76561198312638244,"{'funny': '', 'posted': 'Posted July 21.', 'la...","{'funny': '', 'posted': 'Posted July 10.', 'la...","{'funny': '', 'posted': 'Posted July 10.', 'la...","{'funny': '', 'posted': 'Posted July 8.', 'las...",None,None,None,None,None,None


In [261]:
dfreviews2.to_csv('revies_por_usuario.csv',index=False)

In [256]:
dfreviews2.iloc[0,1]

{'funny': '',
 'posted': 'Posted November 5, 2011.',
 'last_edited': '',
 'item_id': '1250',
 'helpful': 'No ratings yet',
 'recommend': True,
 'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'}